# Masterclass Workshop: SMS Classificatie
Ooit een vervelend SMS berichtje (voor de Generation-Z'ers onder ons: dat is een WhatsApp bericht waar je voor moest betalen) gehad waarin je werd gevraagd om "SERVICE AAN" naar een obscuur nummer te sturen om zo kans te maken op een prijs? Ontzettend vervelend altijd. Jammer genoeg waren er nog maar weinig SMS spam filters in die tijd...

Ondanks SMS allang weer op zijn retour is, gaan we vandaag het probleem van jaren geleden oplossen! Vandaag maken we namelijk een spam filter voor SMS. In het Engels. 

Dat lijkt op het eerste gezicht wellicht niet erg nuttig, maar SMS'jes hebben een heel groot voordeel: ze zijn kort en daardoor ontzettend makkelijk om mee te werken. Bovendien generaliseert het probleem van SMS spam-filters heel goed  naar andere toepassingen. Denk bijvoorbeeld aan een email-spamfilter, een model dat support-tickets in een administratie systeem onder de juiste categorie plaatst of een priorisering van klant berichten maakt. Al deze toepassingen gebruiken dezelfde onderliggende concepten. Wat we tijdens deze workshop leren kunnen we dus direct toepassen op belangrijke use-cases binnen de organisatie ! 


![spam-sms-plaatje](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Mobile_phone_spam_Vi.jpg/640px-Mobile_phone_spam_Vi.jpg)



## Tools inladen

Waar marketeers soms de neiging hebben om Data Science en/of AI te presenteren als een soort magie, iets dat vanzelf werkt en uit zichzelf alle problemen oplost, weten wij wel beter. Data science is een geweldige tool om mooie en moderne producten te maken die vervelend werk uit handen kan nemen en een organisatie efficiënter kan laten werken, maar het vereist wel creativiteit en inzicht. In deze workshop nemen wij je mee in een kort proces van hoe wij een model voor SMS-classificatie zouden maken. 

Omdat wij geen zin hebben om al onze gereedschap zelf te maken gebruiken wij veel vrij beschikbare tools (als timmerman bouw je immers ook niet je eigen hamers: daar is de Gamma voor!). Hieronder importeren wij een aantal van deze datascience tools.

![toolbox image](https://cdn.pixabay.com/photo/2012/04/11/17/29/toolbox-29058_960_720.png)

In [ ]:
# Hier laden we onze gereedschapskist in

import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing

np.random.seed(10)
from sklearn.metrics import accuracy_score
from sklearn import svm
from matplotlib import cm
import graphviz
from sklearn.tree import export_graphviz
from numpy import array

import matplotlib.pyplot as plt
import seaborn as sbn
from matplotlib import cm
from IPython.display import display
from IPython.display import display, Markdown

from pylab import rcParams  # Plaatjes worden wat groter

rcParams["figure.figsize"] = 10, 10

pd.set_option("display.max_colwidth", -1)  # We willen zo veel mogelijk text zien

In [ ]:
def get_message_length(text):
    return len(text)

def contains_www(text):
    return int("www." in text)

def frequency_free(text):
    return len([word for word in text.split(" ") if str.lower(word) == "free"])

def number_of_dots(text):
    return text.count(".")

def contains_personal_word(text):
    personal_words = ['i', 'we', 'you', 'he', 'she'] # <---------- HIER AANVULLEN, lijst van persoonlijke woorden
    words = text.split(' ') # splits het bericht op in woorden
    words = [str.lower(word) for word in words] # zet alle woorden om in kleine letters
    personal_words_in_text = filter(lambda word: word in personal_words, words) # haal alle niet persoonlijke woorden uit bericht
    return len(list(personal_words_in_text)) # tel aantal overgebleven persoonlijke woorden

def number_of_spam_words(text):
    spam_words = ["won", "win", "free", "claim", "mobile", "nokia", "1st", "£", "$"]
    words = text.split(" ")                                                             # splits het bericht op in woorden
    words = [str.lower(word) for word in words]                                         # zet alle woorden om in kleine letters
    spam_words_in_text = filter(lambda word: word in spam_words, words)                 # haal alle niet persoonlijke woorden uit bericht
    return len(list(spam_words_in_text))                                                # tel aantal overgebleven persoonlijke woorden

def plot_svc(x, y, z, model):
    xx, yy = np.meshgrid(np.arange(0, 7, 0.02), np.arange(0, 7, 0.02))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.axis("off")
    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
    plt.scatter(x, y, c=z, cmap=cm.coolwarm, s=100)

def show_features(df):
    display(Markdown("Features: \n - " + " \n - ".join(df.columns)))


## Data Inladen

Zoals gezegd gaan we vandaag werken met een bestaande dataset. Deze dataset bestaat uit twee componenten: een SMS-bericht en een classificatie-label. En classificatie-label geeft aan of het bericht dat we krijgen SPAM of HAM (= geen spam) is. Dit gebeurt vaak in AI: iemand maakt een dataset met voorbeelden en geeft aan welke labels bij de data horen, en we gaan slimme trucjes verzinnen om de computer daarvan te laten leren. Vaak moet er dus iemand zijn die van te voren data labelt voordat de computer dat zelf kan. 

Het is ook mogelijk om alleen maar tekst te gebruiken een de computer zelf labels te laten maken, maar omdat hij niet weet wat er uit moet komen kan dat heel slecht werken. Hij heeft namelijk geen voorbeelden! De computer zou dus kunnen beslissen dat hij wil controleren of een SMS bericht een punt bevat, of naar een bepaald land is gestuurd. Door zelf voorbeelden in te laden geven we de computer een doel. 

Het inladen van de data zelf is vrij eenvoudig. We hebben een bestandje waar de voorbeelden inzitten, en de computer zet dit netjes voor ons om naar een tabel:

In [ ]:
df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv",encoding='latin-1') # laad bestand en maak tabel
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)                   # verwijder nutteloze kolommen
df = df.rename(columns = {'v1': "label", 'v2': 'text'})                                   # hernoem de kolommen naar iets dat wij kunnen begrijpen
df.head()                                                                                 # laat het bovenste stukje van de tabel zien

## Exploratory Data Analysis (EDA)
Wanneer we een model willen leren is het handig om vooraf te weten hoe onze dataset er uit ziet. De informatie die we tijdens deze initiele analyse vergaren kunnen wij gebruiken om:
* Te voorspellen wat haalbare resultaten zullen zijn;
* Uit welke onderdelen het model kan bestaan;
* Wat voor techniek we kunnen gebruiken.

We beginnen met het kijken naar wat haalbare resultaten zullen zijn. We willen alvast een gok maken van de kwaliteit van ons model, en vaststellen wat een absolute ondergrens moet zijn voordat we het goed genoeg vinden. Je kan daarvoor een menselijke maat hanteren (bijvoorbeeld: getrainde professionals kunnen gemiddeld 9 uit de 10 spam berichten goed herkennen) en de computer daarmee laten vergelijken. Dit is wat we voor de pressure cooker hebben gedaan: als mensen rond de 80% goed zitten, willen wij dat tenminste ook kunnen halen. Anders heeft ons model niet zo veel zin.

Vandaag hanteren we echter een andere maatstaf: we gaan kijken naar de inbalans de twee labels (ookwel klassen). Dat is een moeilijke manier van zeggen dat we willen zien hoe veel voorbeelden we voor beide categorieën (`spam` en `ham`) hebben. Hiervoor tellen we het aantal labels van de beide groepen en delen we het door het totaal:

In [ ]:
df.groupby('label').count()/df.shape[0]*100

We kunnen dit ook gemakkelijk grafisch weergeven:

In [ ]:
df.groupby('label').count().plot.pie(y='text', figsize=(5, 5))

>### Opdracht
>Probeer `pie` eens door `bar` en `barh` te vervangen. Welk van de twee heeft je voorkeur, en waarom?

We zien nu dat we voor ongeveer 87% aan goede SMS berichten hebben, en slechts 13.4% aan SPAM.

Hier volgt onze eerste belangrijke conclusie uit: **ons model moet ten minste 87% van de SMS'jes goed voorspellen!** *Als dat niet zo was, konden we namelijk beter alles als HAM aanmerken, en dan zouden we gemiddeld genomen een beter resultaat halen*. Dit is altijd een belangrijke eerste stap van het maken van een model. 


Nu we weten hoe de data verdeeld is, is het handig om een aantal voorbeelden van SPAM en HAM te bekijken. Dat kunnen we gebruiken om zelf inspiratie op de doen om een goed model te kunnen bedenken. Eerst kijken we naar 5 willekeurige goede SMS berichten, daarna naar 5 willekeurige SPAM berichten:

In [ ]:
print("HAM\n--------------------")
print(df[df['label'] == 'ham']['text'].sample(5).values)

In [ ]:
print("SPAM\n-----------------")
print(df[df['label'] == 'spam']['text'].sample(5).values)

Nu we weten wat het verschil tussen de twee groepen is en een beetje gevoel hebben gekregen bij hoe de SMS'jes er uit zien, moeten we gaan nadenken over welke informatie we het model gaan geven. (*De meeste) AI modellen kunnen namelijk niet zonder meer zelf bepalen wat voor informatie ze kunnen gebruiken*. We kunnen dus *niet* direct een SMS berichtje aan een model geven en dan automatisch laten trainen. In plaats daarvan moeten we zelf indicaties (ookwel *features*) maken waarvan we denken dat het model daar wat aan heeft. Vaak zijn dit numerieke of binaire (JA/NEE) waardes.

Deze features gaan we maken aan de hand van de inzichten die we hierboven hebben opgedaan. Uit de voorbeelden konden we al snel een aantal verschillen herkennen:
1. `Spam` bevat vaak getallen, `Ham` niet;
2. `Spam` bevat vaak hoofdletters, `Ham` veel minder;
3. `Spam` berichten zijn vaak redelijk lang, `Ham` vaker niet.
4. `Spam` bevat soms een web adres, terwijl dat in `Ham` minder voorkomt.
5. `Ham` is vaak persoonlijker en bevat woorden als `I` en `you`, waar `spam` dat niet doet.
6. `Spam` bevat vaak "free" om producten te promoten

Natuurlijk zijn dit slechts een paar ideeën op basis van een klein aantal voorbeelden. Daarom gaan we eerst de features uit de text extracten, en daarna testen of onze intuitie klopt.

>### Opdracht
>Voer de cellen opnieuw uit om andere voorbeelden te genereren. Kan je nog meer verschillen tussen `spam` en `ham` vinden? 

>**NB** dit is een echte dataset, en kan *expliciete content* bevatten.

### Digits

Om te beginnen gaan we het aantal getallen (*digits*) in een tekst bericht bepalen en deze toevoegen aan onze dataset. Gelukkig hoeven we dat niet handmatig te doen: daar hebben we de computer voor!

In [ ]:
def count_number_of_digits(text):
    return sum(c.isdigit() for c in text)

df['number_of_digits'] = df['text'].apply(count_number_of_digits)
df.head()

### Hoofdletters

We kunnen nu al zien dat we het ene spam bericht goed kunnen herkennen. We gaan nu hetzelfde doen voor hoofdletters...

In [ ]:
def count_number_of_upper_cases(text):
    return sum(c.isupper() for c in text)

df['number_of_upper_cases'] = df['text'].apply(count_number_of_upper_cases)
df.head()

### Overige features
Nu we hebben gezien hoe we het aantal nummers en aantal hoofdletters kunnen tellen en toevoegen aan de tabel met data, doen we de rest van de features in één keer:

In [ ]:
df['message_length']           = df['text'].apply(get_message_length)
df['contains_www']             = df['text'].apply(contains_www)
df['frequency_free']           = df['text'].apply(frequency_free)
df['number_of_personal_words'] = df['text'].apply(contains_personal_word)
df['number_of_dots']           = df['text'].apply(number_of_dots)
df['number_of_spam_words']     = df['text'].apply(number_of_spam_words)
df.head()

> ### Opdracht

> In de `number_of_personal_words` feature hebben we een aantal woorden opgenomen die aan kunnen duiden dat het om een persoonlijk gesprek gaat. Zoek de functie boven in dit notebook die deze feature maakt, en probeer een aantal woorden toe te voegen die ook een persoonlijk bericht kunnen aanduiden.

Nu we een aantal manieren hebben gevonden om een SMS-bericht te representeren, kunnen we een nieuwe tabel maken waar alleen de informatie in zit die de computer nodig heeft. De tekst van het bericht zelf kunnen we nu weglaten: we hebben daar immers indicatoren voor gemaakt! Ook zetten we de labels `ham` en `spam` om naar getallen. Dit doen we puuur omdat het beter werkt met de modellen die we gaan trainen.


In [ ]:
df_features = df.loc[:, [column for column in df.columns if column is not "text"]]
df_features.head()

In [ ]:
le = preprocessing.LabelEncoder()
labels = le.fit_transform(df['label'])
df_features['label'] = labels
df_features.head()

## Modelleren

Nu we features hebben gemaakt kunnen we beginnen aan het model. Daarvoor hebben we verschillende opties; allemaal met hun eigen voor- en nadelen. Om een indicatie te geven van hoeveel keuze we hebben, dit is slechts een kleine selectie weergegeven als een algemene wegwijzer:

![sklearn cheatsheet](https://scikit-learn.org/stable/_static/ml_map.png)

Om te beginnen volgen we het schema. We heben meer dan 50 rijen aan data, dus we mogen door. We hebben gelabelde data (`spam` of `ham`)  en minder dan 100,000 regels. Het eerste model dat we dan kunnen proberen is een *lineare support vector classifier*.

### Support Vector Classifiers (SVC)

Een lineare support vector machine ziet alle rijen in de tabel als datapunten in een N-dimensionale ruimte en probeert de verschillende groepen te scheiden met een lineare functie. 

Dat klinkt misschien heel wat, maar er zit een heel simpel idee achter. We zien een rij aan getallen als een punt, en proberen de groepen van punten van elkaar te scheiden met een lijn. Hieronder maken we een simpel voorbeeldje. De rode punten zijn `ham`, de blauwe punten zijn `spam`. 

In [ ]:
data = array([[1,  2,  1],
              [2,  5,  1],
              [3,  3,  1],
              [5,  6,  0],
              [6,  1,  0]])

plt.scatter(x=data[:,0], y=data[:,1], c=data[:,2], cmap=cm.coolwarm, s=100)

Als we hier een support vector classifier van willen leren, trekken we een lijn tussen de twee en berekenen we de lineare formule voor de lijn. Deze lijn kunnen we opslaan en altijd weer gebruiken! Als we een willekeurig punt willen classificeren hoeven we zo alleen maar te kijken of die links of rechts van de lijn valt! 

In [ ]:
data = array([[1,  2,  1],
              [2,  5,  1],
              [3,  3,  1],
              [5,  6,  0],
              [6,  1,  0]])

features =  data[:,0:-1]
labels   =  data[:,-1]

model = svm.SVC(gamma='scale',kernel='linear')
model.fit(features, labels)  

plot_svc(features[:,0], features[:,1],labels,model)

Dit gaat helemaal goed zolang onze data precies in twee groepjes op te delen is. Dit is helaas niet het geval voor alle datasets:

In [ ]:
data = array([[1,  2,  1],
              [2,  5,  1],
              [2.5,4,  0],
              [3,  3,  1],
              [5,  6,  0],
              [6,  1,  0]])

features =  data[:,0:-1]
labels   =  data[:,-1]

model = svm.SVC(gamma='scale',kernel='linear')
model.fit(features, labels)  

plot_svc(features[:,0], features[:,1],labels,model)



Nu is het onmogelijk om een rechte lijn te trekken die alle datapunten van elkaar scheidt. We kunnen daar twee oplossingen voor bedenken:
* We beperken ons niet tot rechte (lineare) lijnen maar kijken ook naar kromme lijnen. Hierdoor kunnen we het rode punt tussen de twee blauwe punten meepakken
* We accepteren dat we niet alle punten van elkaar kunnen scheiden en proberen ons verlies te minimaliseren.

Hoewel het eerste punt wellicht beter lijkt (dan houden we immers een perfecte score!) is er meer voor de tweede oplossing te zeggen. In het eerste geval maken we een ontzettend complex model, die wellicht meer fouten gaat maken wanneer we meer data gaan toevoegen. Bovendien wordt alles een stuk onbegrijpelijker. Hier gaat het slechts om één moeilijk punt, maar wat als er duizenden zijn? Dat wordt veel te complex. We kiezen dus meestal voor de tweede aanpak: *we proberen ons verlies te minimaliseren*. 

**Dit is in essentie de basis van machine learning: we kiezen een numerieke representatie voor een dataset en proberen dan, afhankelijk van het model, een scheiding tussen de punten te vinden die het aantal fouten minimaliseerd. **

We zouden nu kunnen beslissen om ons niet tot rechte (lineare) lijnen te beperken, maar ook te kijken naar kromme lijnen (polynomen). Hierdoor kunnen we het blauwe punt tussen de twee rode punten 'meepakken'...

In [ ]:
data = array([[1,  2,  1],
              [2,  5,  1],
              [2.5,4,  0],
              [3,  3,  1],
              [5,  6,  0],
              [6,  1,  0]])

features =  data[:,0:-1]
labels   =  data[:,-1]

model = svm.SVC(gamma='scale',kernel='poly', degree=5)
model.fit(features, labels)  

plot_svc(features[:,0], features[:,1],labels,model)

Op het eerste gezicht lijkt dit misschien een goed idee, maar we hebben nu een ontzettend complex model gemaakt. Bovendien is dit slechts voor één punt, wat als we meer lastige punten hebben....

In [ ]:
data = array([[1, 2, 1],
       [2,   5,   1],
       [2.5, 4,   0],
       [4,   3,   1],
       [4,   1,   1],
       [3.3, 6,   0],
       [5,   3,   0],
       [6,   1,   0],
       [3.5, 4,   1],
       [5,   4.5, 0],
       [6,   6,   1]])

features =  data[:,0:-1]
labels   =  data[:,-1]

model = svm.SVC(gamma='scale',kernel='poly', degree=7) # verander hier de graad van de polynoom
model.fit(features, labels)  

plot_svc(features[:,0], features[:,1],labels,model)

>### Opdracht
Experimenteer hier met:
1. Het weglaten van datapunten (zet een `#` voor een regel met coordinaten)
2. Het veranderen van de graad van de polynoom (verander `degree=7` door bijvoorbeeld `degree=2`)

We hebben nu een ontzettend lastig model getraind dat heel specifiek is, terwijl we maar heel weinig data hebben om dat uit af te leiden. Het zou bijvoorbeeld best kunnen dat er twee datapunten waren die een kleine meetfout hadden, waardoor hele "gebieden" aan blauw onnodig zijn ingetrokken. Misschien heb je dit gezien tijdens het experimenteren. 

We zouden nu beter kunnen overwegen om een andere manier aan te houden: we laten een aantal fouten toe, maar proberen dat te minimaliseren:

In [ ]:
data = array([[1, 2, 1],
       [2, 5, 1],
       [2.5, 4, 0],
       [4, 3, 1],
       [4, 1, 1],
       [3.3, 6, 0],
       [5, 3, 0],
       [6, 1, 0],
       [3.5, 4, 1],
       [5, 4.5, 0],
       [6, 6, 1]])

features =  data[:,0:-1]
labels   =  data[:,-1]

model = svm.SVC(gamma='scale',kernel='linear')
model.fit(features, labels)  

plot_svc(features[:,0], features[:,1],labels,model)

> ### Opdracht
Experimenteer ook hier met het weglaten van punten. Wat voor effect heeft dat op het model? Hoe reageert het nu? Hoe vergelijkt dat met het polynome model? Probeer ook punten toe te voegen op strategische plekken om het model "in de war" te brengen. Lukt dat? 

We hebben nu twee punten fout geclassificeerd, maar ons model ziet er veel schoner uit. Vaak verkiezen we een klein model boven een complex model. Erg complexe modellen lijken vaak wel bij de dataset te passen zoals wij die hebben, maar zodra er een nieuw datapunt bij komt blijkt over het algemeen dat het simpele model veel beter past. Complexe modellen reageren meestal "te heftig" op kleine veranderingen waardoor resultaten in de praktijk erg slecht kunnen uitvallen.

Wat wel blijft opvallen: een SVC kan goed voorspellen zodra de groepjes netjes uit elkaar liggen. Zodra ze te veel door elkaar heen gaan wordt voorspellen lastig. Veel van AI komt hier eigenlijk op neer: we proberen features te vinden die de groepjes van elkaar onderscheiden, en trekken dan een lijn. Soms doen we dat eenvoudig met twee features en een SVC zoals hierboven, soms maken we het wat lastiger en voegen we meer features (en dus meer dimensies) toe. Nu gaan we kijken naar de echte dataset, en proberen we drie features te gebruiken voor het voorspellen. Eerst bekijken we het zelf visueel, en daarna trainen we een model. 

Laten we eerst kijken welke features we hebben gemaakt:

In [ ]:
show_features(df_features)

Nu gaan we inzoomen op drie features: `message_length`, `number_of_spam_words`, en `number_of_dots`:

In [ ]:
import plotly.express as px
fig = px.scatter_3d(df_features, x='message_length', y='number_of_spam_words', z='number_of_dots',color='label')
fig.show()

We zien hier dat de gele punten (spam) gedeeltelijk mooi van de blauwe punten te scheiden zijn. In het midden is het nog een beetje gemixt, maar we kunnen nu een model trainen om te kijken hoe goed we zijn.

Eerst splitten we de dataset op in twee delen: de features (`X`) en de labels (`y`):

In [ ]:
feature_names = ['number_of_spam_words', 'message_length', 'number_of_dots']
X = df_features[feature_names]
y = df_features['label']

Daarna gaan we er voor zorgen dat onze resultaten eerlijk zijn. We maken daarvoor twee datasets:
1. Een _train set_ waarin datapunten zitten waarop we het model gaan fitten/trainen (deze punten gaat de SVC proberen te scheiden met de lijn, zoals eerder)
2. Een _test set_, waarin datapunten zitten waar we het getrainde/gefitte model toepassen. Omdat we de correcte labels weten die bij deze punten horen kunnen we deze dataset gebruiken om te testen hoe goed we zijn. 

> ### Opdracht
Waarom zouden we twee aparte sets maken? Waarom trainen en testen we niet met dezelfde set aan data? 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)

Nadat we de data hebben opgesplitst trainen we de Support Vector Classifier zoals eerder...

In [ ]:
model = svm.SVC(gamma='scale',kernel='linear')
model.fit(X_train, y_train) ;

En gebruiken we het model om de test set te voorspellen. Van de voorspellingen meten we de accuracy:

In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

91% goed!! Dat is al behoorlijk aardig. We begonnen met 87% als we altijd "geen spam" zouden voorspellen, maar we kunnen dit dus nu al beter!

> ### Opdracht
Verander de features in het model en kijk of je de voorspelling hoger krijgt! Je kan features omwisselen, toevoegen en verwijderen.

### Decision Trees

Het vorige model was best goed, maar het heeft toch een groot probleem: het is moeilijk om uit te leggen! Voor een spamfilter is het prima als het antwoord alleen "spam" of "geen spam" is, maar voor hypotheken moeten we de klant kunnen uitleggen wat de òòrzaak van de afwijzing was. "Ja beste klant, u valt nou eenmaal aan de verkeerde kant van een 3D vlak" is weinig klantvriendelijk. Daarom gaan we nu kijken naar een andere vorm van Supervised Machine Learning: Decision Trees. Deze geven juist aan wat de reden op afwijzing of goedkeuring is. 


We beginnen weer met dezelfde setup: we pakken de features die we willen, maken een train- en testset, en trainen het model:

In [ ]:
# Maak de features
feature_names = ['number_of_spam_words', 'message_length', 'number_of_dots']
X = df_features[feature_names]
y = df_features['label']

# Split in train- en test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)

# Maak een beslisboom
clf = DecisionTreeClassifier(max_depth=2) # <---- verander hier de diepte van de boom!
clf = clf.fit(X_train,y_train)

# Voorspel de output
y_pred = clf.predict(X_test)

# Bereken de accuracy
accuracy_score(y_test,y_pred)

We zijn met dezelfde features al een klein beetje beter dan de SVC! Maar het mooiste voordeel komt nog...

In [ ]:
data = export_graphviz(clf,out_file=None,feature_names=feature_names,class_names=['ham', 'spam'],   
                         filled=True, rounded=True,  
                         special_characters=True)
graphviz.Source(data)

... de beslisboom! In dit plaatje staan de regels die gebruikt worden gemaakt om een keuze te maken. We beginnen bovenaan. Is het aantal "spam woorden" groter dan 0.5? Dan gaan we naar rechts. We kijken vervolgens of een bericht meer of minder dan 103 karakters bevat. Hebben we er minder? Dan is het bericht waarschijnlijk geen spam. Bevat het meer karakters? Dan is het waarschijnlijk wel spam! Met een figuur als deze kunnen we relatief makkelijk uitleggen waarom de computer een bepaalde keuze heeft gemaakt. Voor een hypotheek zouden wij bijvoorbeeld kunnen aangeven dat de Loan to Value over de grens heen is. Veel fijner dus!

> ### Opdrachten
1. Verander de diepte van de boom (door `max_depth` te veranderen hierboven). Wat gebeurt er met de accuracy als je de boom dieper maakt? En wat als hij minder diep is? Wordt het begrijpbaarder of niet?
2. Verander de features, hoe hoog kan je de accuracy krijgen? Beter dan de SVC?
3. Welk model vind jij fijner, de SVC of de Decision Tree? En waarom?
4. Wat betekent de "value" waarde in het figuur van de decision tree? Kan je dat achterhalen? Wat zou dit kunnen aangeven?
5. Zoek op wat de `gini-coefficient` is
6. Hoe zou de `gini-cofficient` helpen om een decision tree te maken? 

# Bonus Opdracht

Een ander veelgebruikt model bij Machine Learning is de Naive Bayes Classifier. Er wordt wel eens over gezegd dat het nooit de beste classifier is, maar wel stabiel de een-na-beste keus. Een leuke optie om ook te verkennen dus!

Deze opdracht is iets anders dan de bovenstaande opdrachten: dit keer mag je als echte Data Scientist zelf aan de slag met het vinden van de juiste methodes om het model te maken!

Opdrachten:
1. Waarom heet dit een Naive **Bayes** Classifier? Waar zijn deze modellen op gebaseerd?
2. Wat maakt de "Naive" Bayes Classifier zo "Naive"? 
3. Hoe uitlegbaar zijn NBCs? Hoe verhouden ze zich op dit gebied tot SVC en Decision Trees?
4. Train de NBC op onze bestaande data (*tip: dit hoef je niet helemaal zelf te maken, zoek op scikit-learn, onze standaard tool)*
5. Hoe verhouden de resultaten zich tot die van de andere modellen? Werkt dit beter of slechter? Kan je nog andere verschillen vinden? 

In [ ]:
## Your Code Here...



# Going forward...

Hopelijk heeft deze masterclass je geholpen om een beetje inzicht te krijgen in wat Machine Learning is, wat het kan, en misschien ook hoe het werkt. Je hebt kunnen zien dat ML weinig met magie maar alles met data en getallen te maken heeft. Als laatste opdracht... zie je mogelijkheden voor ML binnen de bank? Hoe en waar zou jij ML toepassen? Wij zijn benieuwd :)

![robot-bye](https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/NAO_waving.JPG/308px-NAO_waving.JPG)